In [1]:
import os
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
# Carregar variáveis de ambiente
load_dotenv()

# Obter variáveis de ambiente
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT', '5432')
database = os.environ.get('DB_NAME')

# Criar string de conexão
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Criar conexão com o banco de dados
engine = create_engine(connection_string)

In [4]:
def load_data_in_chunks(query, engine, chunksize=50000):
    # Lista para armazenar os DataFrames Dask
    df_list = []
    
    # Ler os dados em chunks usando pandas
    for chunk in pd.read_sql(query, engine, chunksize=chunksize):
        # Converter o chunk pandas DataFrame para Dask DataFrame
        ddf_chunk = dd.from_pandas(chunk, npartitions=1)
        df_list.append(ddf_chunk)
    
    # Concatenar todos os Dask DataFrames em um único DataFrame Dask
    df = dd.concat(df_list)
    return df

In [5]:
query = 'SELECT * FROM ds_market'
df = load_data_in_chunks(query, engine)

In [6]:
# Agrupar por 'item' e somar as vendas
item_sales = df.groupby('item')['sales'].sum().compute()

# Converter para um DataFrame pandas para facilitar a manipulação
item_sales = item_sales.reset_index()

In [7]:
# Ordenar em ordem decrescente de vendas
top_items = item_sales.sort_values('sales', ascending=False)

# Exibir os top 10 itens mais vendidos
print("Itens mais vendidos:")
print(top_items.head(10))

Itens mais vendidos:
                   item    sales
1482  SUPERMARKET_3_090  1004721
1501  SUPERMARKET_3_586   920242
2966  SUPERMARKET_3_252   565299
2162  SUPERMARKET_3_555   491287
2933  SUPERMARKET_3_714   396172
2466  SUPERMARKET_3_587   396119
568   SUPERMARKET_3_694   390001
2928  SUPERMARKET_3_226   363082
842   SUPERMARKET_3_202   295983
3046  SUPERMARKET_3_723   284333


In [8]:
# Ordenar em ordem crescente de vendas
bottom_items = item_sales.sort_values('sales', ascending=True)

# Exibir os top 10 itens menos vendidos
print("Itens menos vendidos:")
print(bottom_items.head(10))

Itens menos vendidos:
                     item  sales
2416  HOME_&_GARDEN_2_101    568
1253     ACCESORIES_2_119    660
1885  HOME_&_GARDEN_2_175    718
3025     ACCESORIES_2_084    746
2178  HOME_&_GARDEN_2_005    757
2775     ACCESORIES_2_111    770
1838  HOME_&_GARDEN_2_245    780
2199  HOME_&_GARDEN_2_130    789
171   HOME_&_GARDEN_2_307    796
66       ACCESORIES_2_023    800


In [9]:
# Agrupar por 'store' e somar as vendas
store_sales = df.groupby('store')['sales'].sum().compute().reset_index()

# Ordenar em ordem decrescente de vendas
top_stores = store_sales.sort_values('sales', ascending=False)

# Exibir as top 10 lojas com mais vendas
print("Lojas com mais vendas:")
print(top_stores.head(10))

Lojas com mais vendas:
               store     sales
9            Tribeca  11192410
0  Greenwich_Village   7701312
4            Roxbury   7218243
6           Yorktown   6548196
3      Queen_Village   6432215
1           Back_Bay   6091767
2             Harlem   5688194
8          South_End   5597629
5    Midtown_Village   5152300
7           Brooklyn   4105128


In [10]:
# Agrupar por 'region' e somar as vendas
region_sales = df.groupby('region')['sales'].sum().compute().reset_index()

# Ordenar em ordem decrescente de vendas
top_regions = region_sales.sort_values('sales', ascending=False)

# Exibir as regiões com mais vendas
print("Regiões com mais vendas:")
print(top_regions)

Regiões com mais vendas:
         region     sales
0      New York  28687044
2        Boston  18907639
1  Philadelphia  18132711


In [11]:
# Agrupar por 'category' e somar as vendas
category_sales = df.groupby('category')['sales'].sum().compute().reset_index()

# Ordenar em ordem decrescente de vendas
top_categories = category_sales.sort_values('sales', ascending=False)

# Exibir as categorias com mais vendas
print("Categorias com mais vendas:")
print(top_categories)

Categorias com mais vendas:
        category     sales
2    SUPERMARKET  45114388
0  HOME_&_GARDEN  14486027
1     ACCESORIES   6126979


In [12]:
# Converter a coluna 'date' para datetime
df['date'] = dd.to_datetime(df['date'])

In [13]:
# Criar uma coluna indicando se é um evento
df['is_event'] = df['event'].notnull()

In [14]:
# Coletar as datas dos eventos
event_dates = df[df['is_event']].date.compute().drop_duplicates()

In [17]:
import numpy as np

# Converter as datas dos eventos para um conjunto para busca eficiente
event_dates_set = set(event_dates.dt.date)

# Criar uma coluna 'event_period' categorizando cada data
def get_event_period(date):
    if date.date() in event_dates_set:
        return 'Durante o Evento'
    elif any((date.date() - pd.Timedelta(days=i)) in event_dates_set for i in range(1, 8)):
        return 'Antes do Evento'
    elif any((date.date() + pd.Timedelta(days=i)) in event_dates_set for i in range(1, 8)):
        return 'Após o Evento'
    else:
        return 'Fora do Evento'

df['event_period'] = df['date'].apply(get_event_period)

c:\Users\rpiaz\Desenvolvimento\DSMarket\.venv\Lib\site-packages\dask_expr\_collection.py:4376: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('date', 'object'))

  warnings.warn(meta_warning(meta))


In [18]:
# Agrupar por período de evento e somar as vendas
event_sales = df.groupby('event_period')['sales'].sum().reset_index()

print("Vendas por período de evento:")
print(event_sales)

Vendas por período de evento:
Dask DataFrame Structure:
              event_period  sales
npartitions=1                    
                    object  int64
                       ...    ...
Dask Name: reset_index, 1181 expressions
Expr=ResetIndex(frame=Sum(frame=(Assign(frame=Assign(frame=Assign(frame=Concat(frames=[df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, df, 

In [ ]:
# Ordenar os períodos para exibição lógica
event_sales['event_period'] = pd.Categorical(
    event_sales['event_period'], 
    categories=['Antes do Evento', 'Durante o Evento', 'Após o Evento', 'Fora do Evento'], 
    ordered=True
).astype(str)

# Ensure event_sales is a Pandas DataFrame
if not isinstance(event_sales, pd.DataFrame):
    raise TypeError("event_sales must be a Pandas DataFrame")

# Convert the entire DataFrame to Dask DataFrame
event_sales = dd.from_pandas(event_sales, npartitions=1)

# Sort the Dask DataFrame
event_sales = event_sales.compute().sort_values('event_period')

# Criar um gráfico de barras
plt.bar(event_sales['event_period'], event_sales['sales'])
plt.xlabel('Período em Relação ao Evento')
plt.ylabel('Vendas Totais')
plt.title('Impacto de Eventos nas Vendas')
plt.show()

In [ ]:
# Agrupar as vendas por data
daily_sales = df.groupby('date')['sales'].sum().compute().reset_index()

# Visualizar as vendas diárias
plt.figure(figsize=(14,7))
plt.plot(daily_sales['date'], daily_sales['sales'])
plt.xlabel('Data')
plt.ylabel('Vendas Totais')
plt.title('Vendas Diárias')
plt.show()

In [ ]:
# Adicionar coluna com o número da semana
daily_sales['week'] = daily_sales['date'].dt.isocalendar().week

# Agrupar as vendas por semana
weekly_sales = daily_sales.groupby('week')['sales'].sum().reset_index()

# Visualizar as vendas semanais
plt.figure(figsize=(14,7))
plt.plot(weekly_sales['week'], weekly_sales['sales'])
plt.xlabel('Semana do Ano')
plt.ylabel('Vendas Totais')
plt.title('Vendas Semanais')
plt.show()

# Time Series Analysis

In [ ]:
# Ensure 'date' is in datetime format
df['date'] = dd.to_datetime(df['date'])

# Set 'date' as the index
df = df.set_index('date')

# Resample sales data to get daily total sales
daily_sales = df['sales'].resample('D').sum().compute()

# Plot daily sales
plt.figure(figsize=(14,7))
plt.plot(daily_sales.index, daily_sales.values)
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.title('Daily Total Sales Over Time')
plt.show()

In [ ]:
# Get the list of top 5 items
top_5_items = top_items['item'].head(5).tolist()

# Filter data for top 5 items
df_top_items = df[df['item'].isin(top_5_items)]

# Group by date and item, then sum sales
item_daily_sales = df_top_items.groupby(['date', 'item'])['sales'].sum().compute().reset_index()

# Pivot the data for plotting
item_daily_sales_pivot = item_daily_sales.pivot(index='date', columns='item', values='sales')

# Plot sales trends for top 5 items
plt.figure(figsize=(14,7))
for item in top_5_items:
    plt.plot(item_daily_sales_pivot.index, item_daily_sales_pivot[item], label=item)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Daily Sales Trends for Top 5 Items')
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Since statsmodels requires a pandas Series, ensure the data is in the correct format
daily_sales = daily_sales.asfreq('D')  # Ensure the index is a DateTimeIndex with daily frequency

# Perform seasonal decomposition
decomposition = seasonal_decompose(daily_sales, model='additive')

# Plot the decomposition
decomposition.plot()
plt.show()

In [ ]:
# Compute average sell_price per item per day
avg_price = df.groupby(['date', 'item'])['sell_price'].mean().compute().reset_index()

# Compute total sales per item per day
total_sales = df.groupby(['date', 'item'])['sales'].sum().compute().reset_index()

# Merge the dataframes
price_sales = pd.merge(total_sales, avg_price, on=['date', 'item'])

In [ ]:
# For simplicity, analyze one item
item_to_analyze = top_items['item'].iloc[0]

item_data = price_sales[price_sales['item'] == item_to_analyze]

# Calculate percentage change in price and sales
item_data['price_pct_change'] = item_data['sell_price'].pct_change()
item_data['sales_pct_change'] = item_data['sales'].pct_change()

# Remove rows with NaN values
item_data = item_data.dropna()

# Calculate elasticity
item_data['elasticity'] = item_data['sales_pct_change'] / item_data['price_pct_change']

# Plot elasticity over time
plt.figure(figsize=(14,7))
plt.plot(item_data['date'], item_data['elasticity'])
plt.xlabel('Date')
plt.ylabel('Price Elasticity')
plt.title(f'Price Elasticity Over Time for {item_to_analyze}')
plt.show()

# Average elasticity
average_elasticity = item_data['elasticity'].mean()
print(f"Average price elasticity for {item_to_analyze}: {average_elasticity}")

In [ ]:
# Compute daily total sales
daily_sales_df = df.groupby('date')['sales'].sum().compute().reset_index()

# Create date features
daily_sales_df['day_of_week'] = daily_sales_df['date'].dt.dayofweek
daily_sales_df['month'] = daily_sales_df['date'].dt.month

# Create lag features
daily_sales_df['lag_1'] = daily_sales_df['sales'].shift(1)
daily_sales_df['lag_7'] = daily_sales_df['sales'].shift(7)

# Create rolling features
daily_sales_df['rolling_mean_7'] = daily_sales_df['sales'].rolling(window=7).mean()
daily_sales_df['rolling_std_7'] = daily_sales_df['sales'].rolling(window=7).std()

# Create event indicator
daily_sales_df['is_event'] = daily_sales_df['date'].isin(event_dates.compute()).astype(int)

# Drop rows with NaN values
daily_sales_df = daily_sales_df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

# Features and target
X = daily_sales_df[['day_of_week', 'month', 'lag_1', 'lag_7', 'rolling_mean_7', 'rolling_std_7', 'is_event']]
y = daily_sales_df['sales']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Test RMSE: {rmse}")

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(daily_sales_df['date'].iloc[-len(y_test):], y_test, label='Actual Sales')
plt.plot(daily_sales_df['date'].iloc[-len(y_test):], y_pred, label='Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales')
plt.legend()
plt.show()

In [ ]:
# Compute z-scores
daily_sales_df['sales_zscore'] = (daily_sales_df['sales'] - daily_sales_df['sales'].mean()) / daily_sales_df['sales'].std()

# Identify anomalies
threshold = 3  # Adjust based on your needs
anomalies = daily_sales_df[abs(daily_sales_df['sales_zscore']) > threshold]

print("Anomalous sales days:")
print(anomalies[['date', 'sales', 'sales_zscore']])

In [ ]:
# Assuming df has a 'customer_id' column

# Compute Recency, Frequency, Monetary value
import datetime as dt

# Set analysis date as the last date in the dataset
analysis_date = df['date'].max().compute()

# Compute RFM metrics
rfm = df.groupby('customer_id').agg({
    'date': lambda x: (analysis_date - x.max()).days,
    'id': 'count',
    'sales': 'sum'
}).compute()

rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Assign scores
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=range(5, 0, -1))
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=range(1, 6))
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=range(1, 6))

# Compute RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

print("Customer Segmentation based on RFM:")
print(rfm.head())

In [ ]:
# Compute daily sales per category
category_daily_sales = df.groupby(['date', 'category'])['sales'].sum().compute().reset_index()

# Pivot the data
category_sales_pivot = category_daily_sales.pivot(index='date', columns='category', values='sales').fillna(0)

# Compute correlation matrix
correlation_matrix = category_sales_pivot.corr()

print("Correlation between categories:")
print(correlation_matrix)

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Categories')
plt.show()

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
output_notebook()

# Prepare data
source = ColumnDataSource(data={
    'date': daily_sales_df['date'],
    'sales': daily_sales_df['sales']
})

# Create a figure
p = figure(x_axis_type='datetime', title='Daily Sales', plot_height=350, plot_width=800)
p.line('date', 'sales', source=source)
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Sales'

# Show the plot
show(p)

In [ ]:
# Convert data types to more efficient ones
df['sales'] = df['sales'].astype('int32')
df['sell_price'] = df['sell_price'].astype('float32')

In [ ]:
# Example: Only load data from the last 2 years
query = "SELECT * FROM ds_market WHERE date >= '2021-01-01'"
df = load_data_in_chunks(query, engine)